In [1]:
import pandas as pd

payload = pd.read_pickle("outputs/simulation_payload.pkl")

candles = payload["candles"].copy()
candles["datetime"] = pd.to_datetime(candles["datetime"])
candles = candles.set_index("datetime")

closed_trades = payload["closed_trades"].copy()
if len(closed_trades) > 0:
  closed_trades["entry_time"] = pd.to_datetime(closed_trades["entry_time"])
  closed_trades["exit_time"] = pd.to_datetime(closed_trades["exit_time"])

open_trades = payload["open_trades"].copy()
if not open_trades.empty:
  open_trades["entry_time"] = pd.to_datetime(open_trades["entry_time"])

# Funciones

In [2]:
import pandas as pd
import plotly.graph_objects as go
from typing import Optional


def plot_candles_and_trades(
    candles: pd.DataFrame,
    trades: Optional[pd.DataFrame] = None,
    show: bool = True,
    skip_weekends: bool = True,
):
  """
    Build an interactive candlestick chart and overlay closed trades if provided.

    candles: df indexed by datetime with open, high, low, close[, volume]
    trades: optional df with columns:
      side ['BUY'|'SELL'], entry_time, exit_time,
      entry_price, exit_price, take_profit, stop_loss
    """
  cdl = candles.copy().sort_index()
  if not isinstance(cdl.index, pd.DatetimeIndex):
    cdl.index = pd.to_datetime(cdl.index, utc=True, errors="coerce")

  fig = go.Figure()
  fig.add_trace(
      go.Candlestick(
          x=cdl.index,
          open=cdl["open"],
          high=cdl["high"],
          low=cdl["low"],
          close=cdl["close"],
          name="OHLC",
      )
  )

  if trades is not None and not trades.empty:
    trs = trades.copy()
    trs["entry_time"] = pd.to_datetime(
        trs["entry_time"], utc=True, errors="coerce"
    )
    trs["exit_time"] = pd.to_datetime(
        trs["exit_time"], utc=True, errors="coerce"
    )
    for _, r in trs.iterrows():
      et = r["entry_time"]
      xt = r["exit_time"]
      ep = float(r["entry_price"])
      xp = float(r["exit_price"])
      tp = float(r["take_profit"]) if pd.notna(r.get("take_profit")) else None
      sl = float(r["stop_loss"]) if pd.notna(r.get("stop_loss")) else None

      fig.add_trace(
          go.Scatter(
              x=[et],
              y=[ep],
              mode="markers",
              name="Entry",
              marker_symbol="triangle-up",
              marker_size=10,
              hovertext=[f'Entry {r.get("side","")}'],
              hoverinfo="text",
          )
      )
      fig.add_trace(
          go.Scatter(
              x=[xt],
              y=[xp],
              mode="markers",
              name="Exit",
              marker_symbol="triangle-down",
              marker_size=10,
              hovertext=[f'Exit ({r.get("result","")})'],
              hoverinfo="text",
          )
      )
      fig.add_trace(
          go.Scatter(
              x=[et, xt],
              y=[ep, xp],
              mode="lines",
              name="Trade",
              hoverinfo="skip",
          )
      )
      if tp is not None:
        fig.add_trace(
            go.Scatter(
                x=[et, xt],
                y=[tp, tp],
                mode="lines",
                name="TP",
                line=dict(dash="dash"),
                hoverinfo="skip",
            )
        )
      if sl is not None:
        fig.add_trace(
            go.Scatter(
                x=[et, xt],
                y=[sl, sl],
                mode="lines",
                name="SL",
                line=dict(dash="dot"),
                hoverinfo="skip",
            )
        )

  fig.update_layout(
      xaxis_rangeslider_visible=False,
      hovermode="x unified",
      legend_title_text="Legend",
  )
  if skip_weekends:
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
  if show:
    fig.show()
  return fig


In [3]:
import pandas as pd
from zoneinfo import ZoneInfo
from datetime import time
from tradeo.ohlc import OHLC
from tradeo.trading_methods import calculate_poc_vah_val, calculate_heikin_ashi
from sorul_tradingbot.strategy.private.volume_12 import Volume
from sorul_tradingbot.strategy.simulator.simulator import SimulatedMTClient


def _get_session_times(now_dt: pd.Timestamp) -> tuple[time, time]:
  """Replica la lógica de Volume._get_session_times."""
  ny_tz = ZoneInfo("America/New_York")
  utc_tz = ZoneInfo("UTC")

  if now_dt.tzinfo is None:
    aware_now = now_dt.tz_localize(utc_tz)
  else:
    aware_now = now_dt.tz_convert(utc_tz)

  now_ny = aware_now.astimezone(ny_tz)
  dst = now_ny.dst()
  is_dst = bool(dst and dst.total_seconds())
  if is_dst:
    return time(14, 30), time(20, 0)
  return time(15, 30), time(21, 0)


def compute_daily_levels(candles: pd.DataFrame) -> pd.DataFrame:
  """Devuelve un DataFrame con poc/vah/val por sesión."""
  if candles.empty:
    return pd.DataFrame(columns=["session_date", "POC", "VAH", "VAL"])

  idx = pd.to_datetime(candles.index)
  if getattr(idx, "tz", None) is None:
    idx = idx.tz_localize("UTC")
  else:
    idx = idx.tz_convert("UTC")
  candles_utc = candles.copy()
  candles_utc.index = idx

  ohlc = OHLC(candles_utc, volume_column_name="volume")
  ohlc = calculate_heikin_ashi(ohlc)

  # usamos cualquier timestamp para obtener el rango horario correcto (DST o no)
  sample_ts = candles_utc.index[-1].to_pydatetime()
  volume_strategy = Volume(SimulatedMTClient())
  start_t, end_t = volume_strategy._get_session_times(sample_ts)

  daily_levels = calculate_poc_vah_val(
      ohlc,
      session_start=start_t.strftime("%H:%M"),
      session_end=end_t.strftime("%H:%M"),
  )
  levels_df = (
      pd.DataFrame(daily_levels).T.reset_index().rename(
          columns={"index": "session_date"}
      )
  )
  levels_df["session_date"] = pd.to_datetime(levels_df["session_date"])
  return levels_df[["session_date", "POC", "VAH", "VAL"]]


# Ejecuciones

In [4]:
import pandas as pd
import plotly.graph_objects as go

levels_df = compute_daily_levels(candles)

fig = plot_candles_and_trades(candles, closed_trades, show=False)

candles_idx = pd.to_datetime(candles.index)
if getattr(candles_idx, "tz", None) is None:
  candles_idx = candles_idx.tz_localize("UTC")
else:
  candles_idx = candles_idx.tz_convert("UTC")
candles_utc = candles.copy()
candles_utc.index = candles_idx

vah_first, val_first = True, True
for _, row in levels_df.iterrows():
  session_date = pd.Timestamp(row["session_date"]).tz_localize("UTC")
  start_time, end_time = _get_session_times(session_date)

  start_dt = session_date.replace(
      hour=start_time.hour, minute=start_time.minute, second=0
  )
  end_dt = session_date.replace(
      hour=end_time.hour, minute=end_time.minute, second=0
  )
  if end_dt <= start_dt:
    end_dt += pd.Timedelta(days=1)

  mask = (candles_utc.index >= start_dt) & (candles_utc.index <= end_dt)
  if not mask.any():
    continue

  x0, x1 = candles_utc.index[mask][[0, -1]]
  vah = row["VAH"]
  val = row["VAL"]

  fig.add_trace(
      go.Scatter(
          x=[x0, x1],
          y=[vah, vah],
          mode="lines",
          line=dict(color="#1f3c88", width=2),
          name="VAH",
          legendgroup="VAH",
          showlegend=vah_first,
          hoverinfo="x+y",
      )
  )
  fig.add_trace(
      go.Scatter(
          x=[x0, x1],
          y=[val, val],
          mode="lines",
          line=dict(color="#6ecff6", width=2),
          name="VAL",
          legendgroup="VAL",
          showlegend=val_first,
          hoverinfo="x+y",
      )
  )
  vah_first = False
  val_first = False


fig.show()
